In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from main.schlogModel import schloglModel

In [ ]:
# Define Schlogl model
smodel = schloglModel()

# Define and set parameters
concA = 10.
concB = 20.
k1 = 6.
k2 = 1.0
k3 = 230.
k4 = 1000.
vol = 8.

smodel.setModelParameters(concA, concB, k1, k2, k3, k4, vol)

In [ ]:
# Define data parameters
domain = [0, 500]
dt = 0.0001
stride = 1
timesteps = 2
data_size = 2560
data_multiplier = 1
out_resolution = 50

smodel.setDataParameters(domain[0], domain[1], dt, stride, timesteps, data_size, data_multiplier, out_resolution)

In [ ]:
# Simulation setup
tstride = 1 # For feeding data into algorithm   

# network settings
WIDTH = 40 #50 #40 #50 #40 #20 #15 #35 #20              # number of neurons per hidden layer
DEPTH = 15 #25 #15 #30 #15 #10 #10 #10 #6               # number of hidden layers
ACTIVATION = tf.nn.relu #tf.sigmoid #hidden layer activation function

# training settings
INIT_L_RATE  = 1e-4 #2e-1
FINAL_L_RATE = 1e-5 #1e-3
batch_size   = 128 #64 #2048
num_batches = int(data_size*data_multiplier/batch_size)
num_epochs = 200

# Other
outscale = (domain[1] - domain[0])/out_resolution
filename = "data/schlogl_data_vol" + str(vol) + "_ndata" + str(data_size) + ".dat"

In [ ]:
# Generate network structure

# Inputs and targets placeholders for trained data
networkInput = tf.placeholder(dtype=tf.float32, shape=(None,2), name='input')
networkTarget = tf.placeholder(dtype=tf.float32, shape=(None, out_resolution), name='target')

# Hidden layers
hidden = (DEPTH-1)*[None]
hidden[0] = networkInput
#next_layer = networkInput
for l in range(DEPTH-2):
    #without skip connections
    hidden[l+1] = tf.layers.dense(hidden[l], WIDTH, activation=ACTIVATION)
    
# Add predition outermost layer
networkPrediction = tf.layers.dense(hidden[DEPTH-2], out_resolution, activation=None, name='prediction')

In [ ]:
# Training procedure

# Define loss function and error
loss = tf.reduce_mean(tf.square(networkPrediction - networkTarget))
error = tf.reduce_max(tf.abs(networkPrediction - networkTarget))

# use stochastic gradient descent with ADAM during optimization
step = tf.train.AdamOptimizer(INIT_L_RATE).minimize(loss)

# # Or use decaying learning rate with SGD
# NUM_ITER = num_batches*batch_size*num_epochs
# global_step = tf.Variable(0, trainable=False)
# learning_rate = tf.train.exponential_decay(
#     INIT_L_RATE,
#     global_step,
#     1,
#     np.exp(np.log(FINAL_L_RATE/INIT_L_RATE) / NUM_ITER),
#     staircase=True
# )
# step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [ ]:
# Print all parameters
print('\n----------------------------------------------------') 
print(' RUNNING EXPERIMENT WITH THE FOLLOWING PARAMETERS: ')
print('----------------------------------------------------\n')
print('depth:\t\t\t{}'.format(DEPTH))
print('width:\t\t\t{}'.format(WIDTH))
print('number of neurons:\t{}'.format(2+(DEPTH-2)*WIDTH))
print('number of connections:\t{}'.format(1+(DEPTH-2)*WIDTH*2+WIDTH*WIDTH*(DEPTH-3)*(DEPTH-2)//2))
print('activation:\t\t{}'.format(ACTIVATION.__name__))
print('learning rate:\t\t{} to {}'.format(INIT_L_RATE, FINAL_L_RATE))
print('epochs: \t\t{}'.format(num_epochs))
print('batch size:\t\t{}'.format(batch_size))
print('\n\n')

In [ ]:
# Generate and/or load data
generateData = False
if generateData:
    smodel.generateData()
inputData, targetData = smodel.loadData(filename)

In [ ]:
# Start Tensorflow session and initialize all network variables
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
# Run the training

# run gradient descent steps with Adam
print('\nStarted training...')
print('{:8s}\t{:8s}\t{:8s}\t{:8s}'.format('Epoch', 'Iteration', 'l2-loss', 'linf-err'))
print('{:8s}\t{:8s}\t{:8s}\t{:8s}'.format(*(4*[8*'-'])))
for epoch in range(num_epochs):
    # generate random batch of inputs and corresponding target values
    indicesList = np.random.choice(data_size*data_multiplier, [num_batches, batch_size], replace = False)
    # Loop over all possible batches within the dataset
    for jbatch in range(num_batches):
        indices =  (indicesList[jbatch]).astype(int)
        inputBatch = np.take(inputData, indices, axis=0)
        #tstep = np.random.randint(0, int(timesteps/tstride), batch_size) #dt*j*tstride*np.ones(len(inputx))
        #time = tstep #dt*tstep*tstride
        #inputBatch = np.reshape(np.column_stack((inputx,time)), [-1,2])
        #inputBatch = np.reshape(inputx, [-1,1])
        
        #currentStepsData = np.take(targetData, tstep, axis=0)
        #matrix = np.take(currentStepsData, indices, axis = 1)
        #targetBatch = np.diagonal(matrix, axis1 = 0, axis2 = 1).transpose()
        #targetBatch = terminalCondition(inputx)
        targetBatch = np.take(np.array(targetData), indices, axis = 0)
        
        
        # take gradient descent step and compute loss & error
        loss_val, error_val, _ = session.run(
            [loss, error, step],
            feed_dict={networkInput: inputBatch, networkTarget: targetBatch}
        )
#         # Loop over all timesteps used for training
#         for tstep in range(int(timesteps/tstride)):
#             time = tstep*np.ones(batch_size) #dt*tstep*tstride*np.ones(BATCH_SIZE)
#             inputBatch = np.reshape(np.column_stack((inputx,time)), [-1,2])
#             targetBatch = np.take(np.array(targetData[tstep]), indices, axis = 0)

#             # take gradient descent step and compute loss & error
#             loss_val, error_val, _ = session.run(
#                 [loss, error, step],
#                 feed_dict={networkInput: inputBatch, networkTarget: targetBatch}
#             )
        if (epoch*num_batches + jbatch) % 500 == 0:
            print('{:8d}\t{:8d}\t{:1.5e}\t{:1.5e}'.format(epoch, epoch*num_batches + jbatch, loss_val, error_val))
print('...finished training.\n')

# tstep = np.random.randint(0, int(timesteps/tstride), BATCH_SIZE) #dt*j*tstride*np.ones(len(inputx))
# currentStepsData = np.take(targetData, tstep, axis=0)
# matrix = np.take(currentStepsData, indices, axis = 1)
# targetBatch = np.diagonal(matrix, axis1 = 0, axis2 = 1).transpose()

In [ ]:
# Plot for vector case

plt.rcParams['figure.figsize'] = (11,8)
# generate full sample grid of input domain
RESOLUTION = 500
xgrid = np.linspace(domain[0] + 1, domain[1], num=RESOLUTION)
#xgrid = xgrid.astype(int)
time = 0*np.ones(len(xgrid))
input_test_batch = np.reshape(np.column_stack((xgrid,time)) , [-1,2])
#input_test_batch = np.reshape(xgrid, [-1,1])


# get model predictions
prediction_test_batch = session.run( networkPrediction, feed_dict={networkInput: input_test_batch})

# Remove negative entries and renormalize
x0index = 50
#renormalized_output = np.mean(prediction_test_batch, axis=0)
renormalized_output = prediction_test_batch[x0index]
renormalized_output[renormalized_output<0] = 0.0
renormalized_output = renormalized_output/np.sum(renormalized_output)

# # Calculate steady state analytically
# n=np.linspace(domain[0], (domain[1]-domain[0])-1, domain[1]-domain[0])
# ss_solution=np.zeros(len(n))
# for i in range(len(n)):
#     ss_solution[i] = smodel.steadystate_solution(i)

# ss_solution = ss_solution/np.sum(ss_solution) 

# plot resultiung histogram
#plt.bar(np.arange(OUTPUT_RES),np.mean(prediction_test_batch, axis=0))
plt.bar(np.arange(outscale/2,outscale*out_resolution,outscale),renormalized_output/outscale, 
        width=outscale, label="NN", color=(0.0, 0.4, 1.0, 0.5))

#Plot analytic solution
# plt.plot(n,ss_solution, '-r', lw = 3, label="Steady state (exact)")
#plt.plot(n,np.log(ss_solution))

#plt.ylim([0.0,0.05])
#plt.xlim([0, outscale*OUTPUT_RES])
plt.xlim([0, 400])
#plt.ylim([0.0,0.02])


plt.ylabel('Probability', fontsize = 35)
plt.xlabel('$X[T]$', fontsize = 35)
plt.legend(fontsize = 35)
plt.tick_params(labelsize=30)
plt.locator_params(axis='y', nbins=6)


plt.show()

In [ ]:
netInput = np.reshape(np.column_stack((240,0)) , [-1,2]) 
prediction = session.run( networkPrediction, feed_dict={networkInput: netInput})
plt.bar(np.arange(OUTPUT_RES),prediction[0])


In [ ]:
def interactive_output(x,t):
    netInput = np.reshape(np.column_stack((x,t)) , [-1,2]) 
    prediction = session.run( networkPrediction, feed_dict={networkInput: netInput})
    # Remove negative entries and renormalize
    #renormalized_output = np.mean(prediction_test_batch, axis=0)
    prediction[prediction<0] = 0.0
    prediction = prediction/np.sum(prediction)

    plt.bar(np.arange(1,outscale*OUTPUT_RES,outscale),prediction[0]/outscale, width=outscale)
    plt.plot(n,ss_solution, '-r')
    plt.xlim([DOMAIN[0], DOMAIN[1]-100])
    plt.ylim([0, 0.05])
    plt.ylabel('Histogram')
    plt.show()

interactive_plot = interactive(interactive_output, x=(DOMAIN[0],DOMAIN[1]-100,10), t=(0,100,10))
output = interactive_plot.children[-1]
interactive_plot

## TESTS

In [ ]:
len([None]*int(timesteps/tstride))

In [ ]:
targetData[0]

In [ ]:
plt.plot(np.mean(targetData, axis=0))

In [ ]:
x0 = 20
A = 1
B = 1
kk1 = 3*x0
kk2 = 1.0
kk3 = x0**3 - 200*x0
kk4 = (3*x0**2-200)
xmax = 100
res = 200
xx = np.linspace(-xmax,xmax,res)
y = ODE_func(xx,kk1,kk2,kk3,kk4,A,B)
plt.plot(xx,y)
plt.ylim([-5000,5000])
plt.xlim([0,50])
plt.plot(xx,0*y,'--k')

In [ ]:
print(kk1,kk2,kk3,kk4)

In [ ]:
4000/20

In [ ]:
x0 = 15
A = 10
B = 20
kk1 = 6
kk2 = 1.0
kk3 = 250
kk4 = 1000
xmax = 100
res = 200
xx = np.linspace(-xmax,xmax,res)
y = ODE_func(xx,kk1,kk2,kk3,kk4,A,B)
plt.plot(xx,y)
plt.ylim([-5000,5000])
plt.xlim([0,50])
plt.plot(xx,0*y,'--k')

In [ ]:
1875/75

In [ ]:
print(kk1,kk2,kk3,kk4)

In [ ]:
xmax = 100
x0 = 50.0
xx = np.linspace(-xmax,xmax,res)
plt.ylim([-5000,5000])
plt.plot(xx,-(xx-x0)**3+(xx-x0))

In [ ]:
numReactions1 = np.random.poisson(lambda1(20)*dt, 1)
numReactions2 = np.random.poisson(lambda2(20)*dt, 1)
numReactions3 = np.random.poisson(lambda3(20)*dt, 1)
numReactions4 = np.random.poisson(lambda4(20)*dt, 1)

In [ ]:
print(numReactions1, numReactions2, numReactions3, numReactions4)

In [ ]:
prediction_test_batch[0]

In [ ]:
n=np.linspace(0,199,200)
ss_solution=np.zeros(200)
for i in range(200):
    ss_solution[i] = steadystate_solution(i)

ss_solution = ss_solution/np.sum(ss_solution) 
plt.xlim([0,200])
#plt.plot(n,np.log(ss_solution))
plt.plot(n,ss_solution)

In [ ]:
N = OUTPUT_RES
plt.bar(np.arange(N)*outscale,np.mean(targetData,axis=0),width=outscale)
# Calculate steady state analytically
n=np.linspace(DOMAIN[0], (DOMAIN[1]-DOMAIN[0])-1, N)
ss_solution=np.zeros(N)
for i in range(N):
    ss_solution[i] = steadystate_solution(int(n[i]))

ss_solution = ss_solution/np.sum(ss_solution) 
plt.plot(n,ss_solution, '-r')
plt.show()

In [ ]:
# Define parameters
concA = 1
concB = 2
k1 = 3.0
k2 = 0.6
k3 = 0.25
k4 = 2.95
vol = 30 #100.0
def func(x):
    return k1*concA*x**2 - k2*x**3 -k4*x + k3*concB

In [ ]:
x = np.linspace(0,50,500)
plt.plot(x,func(x))

In [ ]:
xx=np.float32(np.random.randint( DOMAIN[0], DOMAIN[1], datasize ))

In [ ]:
xx[1]